In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ruta_archivo1 = "Datasets/Vehicle Fuel Economy Data.csv"
df = pd.read_csv(ruta_archivo1)

df.head()

/tmp/ipykernel_26086/526747422.py:2: DtypeWarning: Columns (73,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_archivo1)


,Year,Manufacturer,Model,barrels08,barrelsA08,charge240,city08,city08U,cityA08,cityA08U,...,mfrCode,c240Dscr,charge240b,c240bDscr,createdOn,modifiedOn,startStop,phevCity,phevHwy,phevComb
0,2017,BMW,M6 Coupe,17.500588,0.0,0.0,15.0,14.0,0.0,0.0,...,BMX,NaN,0.0,NaN,04-04-2016,09-09-2016,Y,0.0,0.0,0.0
1,2017,BMW,X3 sDrive28i,12.396250,0.0,0.0,21.0,21.0,0.0,0.0,...,BMX,NaN,0.0,NaN,04-04-2016,22-11-2016,Y,0.0,0.0,0.0
2,2016,Infiniti,Q50S Red Sport,13.523182,0.0,0.0,20.0,19.0,0.0,0.0,...,NSX,NaN,0.0,NaN,04-04-2016,26-09-2016,N,0.0,0.0,0.0
3,2017,MINI,John Cooper Works Convertible,11.018889,0.0,0.0,24.0,23.0,0.0,0.0,...,BMX,NaN,0.0,NaN,12-04-2016,16-09-2016,Y,0.0,0.0,0.0
4,2017,Kia,Sedona SX,14.167143,0.0,0.0,18.0,18.0,0.0,0.0,...,KMX,NaN,0.0,NaN,12-04-2016,02-12-2016,N,0.0,0.0,0.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46186 entries, 0 to 46185
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Year             46186 non-null  int64  
 1   Manufacturer     46186 non-null  object 
 2   Model            46176 non-null  object 
 3   barrels08        46169 non-null  float64
 4   barrelsA08       46169 non-null  float64
 5   charge240        46169 non-null  float64
 6   city08           46169 non-null  float64
 7   city08U          46169 non-null  float64
 8   cityA08          46169 non-null  float64
 9   cityA08U         46169 non-null  float64
 10  cityCD           46169 non-null  float64
 11  cityE            46169 non-null  float64
 12  cityUF           46169 non-null  float64
 13  co2              46169 non-null  float64
 14  co2A             46169 non-null  float64
 15  co2TailpipeAGpm  46169 non-null  float64
 16  co2TailpipeGpm   46169 non-null  float64
 17  comb08      

## I Eliminar columnas que no vamos a utilizar

In [4]:
# Las columnas que vamos a dejar en el dataset por su importancia para nuestro objetivo
columns_to_keep_corrected = [
    'Year', 'Manufacturer', 'Model', 'VClass', 'fuelType', 'fuelType1', 
    'city08', 'highway08', 'comb08', # MPG en ciudad, carretera y combinado para economía de combustible convencional
    'cityA08', 'highwayA08', 'combA08', # MPG en ciudad, carretera y combinado para economía de combustible alternativo
    'co2', 'co2TailpipeGpm', # Emisiones de CO2
    'phevBlended', # Si el vehículo es híbrido enchufable
    'range', # Rango eléctrico para vehículos eléctricos
    'fuelCost08', # Costo anual estimado de combustible convencional
    'fuelCostA08', # Costo anual estimado de combustible alternativo
]

# Filtramos el dataframe para mantener solo las columnas seleccionadas
df = df[columns_to_keep_corrected]

df.head()


,Year,Manufacturer,Model,VClass,fuelType,fuelType1,city08,highway08,comb08,cityA08,highwayA08,combA08,co2,co2TailpipeGpm,phevBlended,range,fuelCost08,fuelCostA08
0,2017,BMW,M6 Coupe,Subcompact Cars,Premium,Premium Gasoline,15.0,22.0,17.0,0.0,0.0,0.0,519.0,519.0,False,0.0,3850.0,0.0
1,2017,BMW,X3 sDrive28i,Small Sport Utility Vehicle 2WD,Premium,Premium Gasoline,21.0,28.0,24.0,0.0,0.0,0.0,374.0,374.0,False,0.0,2700.0,0.0
2,2016,Infiniti,Q50S Red Sport,Midsize Cars,Premium,Premium Gasoline,20.0,26.0,22.0,0.0,0.0,0.0,402.0,402.0,False,0.0,2950.0,0.0
3,2017,MINI,John Cooper Works Convertible,Minicompact Cars,Premium,Premium Gasoline,24.0,32.0,27.0,0.0,0.0,0.0,327.0,327.0,False,0.0,2400.0,0.0
4,2017,Kia,Sedona SX,Minivan - 2WD,Regular,Regular Gasoline,18.0,25.0,21.0,0.0,0.0,0.0,433.0,433.0,False,0.0,2550.0,0.0


## II Exploración y tratamiento de Nulos

In [5]:
# Filtramos el dataset para incluir categorías que corresponden a sedanes y SUVs
sedan_suv_categories = [
    'Compact Cars', 'Large Cars', 'Midsize Cars', 'Subcompact Cars',
    'Small Sport Utility Vehicle 2WD', 'Small Sport Utility Vehicle 4WD',
    'Standard Sport Utility Vehicle 2WD', 'Standard Sport Utility Vehicle 4WD',
    'Sport Utility Vehicle - 2WD', 'Sport Utility Vehicle - 4WD'
]

df = df[df['VClass'].isin(sedan_suv_categories)]

# Verificamos las primeras filas del nuevo dataset filtrado para confirmar el filtro
df.head()


,Year,Manufacturer,Model,VClass,fuelType,fuelType1,city08,highway08,comb08,cityA08,highwayA08,combA08,co2,co2TailpipeGpm,phevBlended,range,fuelCost08,fuelCostA08
0,2017,BMW,M6 Coupe,Subcompact Cars,Premium,Premium Gasoline,15.0,22.0,17.0,0.0,0.0,0.0,519.0,519.0,False,0.0,3850.0,0.0
1,2017,BMW,X3 sDrive28i,Small Sport Utility Vehicle 2WD,Premium,Premium Gasoline,21.0,28.0,24.0,0.0,0.0,0.0,374.0,374.0,False,0.0,2700.0,0.0
2,2016,Infiniti,Q50S Red Sport,Midsize Cars,Premium,Premium Gasoline,20.0,26.0,22.0,0.0,0.0,0.0,402.0,402.0,False,0.0,2950.0,0.0
6,2017,Ford,Escape FWD,Small Sport Utility Vehicle 2WD,Regular,Regular Gasoline,22.0,29.0,25.0,0.0,0.0,0.0,357.0,357.0,False,0.0,2100.0,0.0
8,2017,Lincoln,MKC AWD,Small Sport Utility Vehicle 4WD,Regular,Regular Gasoline,19.0,25.0,22.0,0.0,0.0,0.0,404.0,404.0,False,0.0,2400.0,0.0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27043 entries, 0 to 46185
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            27043 non-null  int64  
 1   Manufacturer    27043 non-null  object 
 2   Model           27043 non-null  object 
 3   VClass          27043 non-null  object 
 4   fuelType        27043 non-null  object 
 5   fuelType1       27043 non-null  object 
 6   city08          27043 non-null  float64
 7   highway08       27043 non-null  float64
 8   comb08          27043 non-null  float64
 9   cityA08         27043 non-null  float64
 10  highwayA08      27043 non-null  float64
 11  combA08         27043 non-null  float64
 12  co2             27043 non-null  float64
 13  co2TailpipeGpm  27043 non-null  float64
 14  phevBlended     27043 non-null  object 
 15  range           27043 non-null  float64
 16  fuelCost08      27043 non-null  float64
 17  fuelCostA08     27043 non-null  floa

## III Tratamiento outliers y duplicados

- Economía de Combustible (city08, highway08, comb08): Los valores atípicos en estas columnas indican vehículos con una eficiencia de combustible excepcionalmente alta o baja en ciudad, carretera, y combinado, respectivamente. Valores muy altos (como 97.0, 100.0, y 98.0 mpg) sugieren la presencia de vehículos eléctricos o híbridos altamente eficientes, mientras que valores más bajos indican vehículos menos eficientes, probablemente con motores de gran tamaño o bajo rendimiento.

- Economía de Combustible Alternativo (cityA08, highwayA08, combA08): Estos valores atípicos sugieren variaciones en la eficiencia de combustibles alternativos. Estos valores pueden representar vehículos eléctricos (EV), híbridos enchufables (PHEV), o vehículos con otros tipos de combustibles alternativos que ofrecen diferentes grados de eficiencia.

- Emisiones de CO2 (co2TailpipeGpm): Los outliers aquí varían desde cero, indicando vehículos eléctricos que no emiten CO2 por el tubo de escape, hasta valores extremadamente altos que sugieren vehículos muy ineficientes desde el punto de vista de las emisiones de CO2.

- Rango (range): Los outliers en el rango indican vehículos eléctricos con capacidades de batería excepcionales, ofreciendo una autonomía muy por encima de la media.

- Costo de Combustible (fuelCost08, fuelCostA08): Los outliers en el costo anual de combustible (tanto convencional como alternativo) sugieren vehículos que son notablemente más caros o más económicos de operar. Los costos extremadamente altos pueden estar asociados con baja eficiencia de combustible y altas emisiones, mientras que los costos bajos indican vehículos eficientes.

In [7]:
# Seleccionamos solo las columnas numéricas
df_numeric = df.select_dtypes(include=['float64', 'int64'])

# Calcula el rango intercuartílico (IQR) para cada columna numérica
Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1

# Identifica los outliers
outliers = ((df_numeric < (Q1 - 1.5 * IQR)) | (df_numeric > (Q3 + 1.5 * IQR)))

# Muestra los outliers
for column in outliers.columns:
    if outliers[column].any():
        print(f"Outliers in {column}:")
        print(df_numeric[column][outliers[column]])
        print()


Outliers in city08:
20       97.0
119      32.0
160      54.0
175      40.0
212      33.0
         ... 
45631    37.0
45909    38.0
46142    38.0
46143    43.0
46163    33.0
Name: city08, Length: 1103, dtype: float64

Outliers in highway08:
20       100.0
160       50.0
232       43.0
238       43.0
239      122.0
         ...  
45628     41.0
45631     45.0
45909     44.0
46142     45.0
46143     52.0
Name: highway08, Length: 933, dtype: float64

Outliers in comb08:
20       98.0
160      52.0
175      40.0
232      40.0
238      36.0
         ... 
45628    36.0
45631    40.0
45909    40.0
46142    41.0
46143    47.0
Name: comb08, Length: 963, dtype: float64

Outliers in cityA08:
35       15.0
53       12.0
81       11.0
109      15.0
110      14.0
         ... 
42518    13.0
42528     9.0
44322    12.0
44512    18.0
44732    13.0
Name: cityA08, Length: 1111, dtype: float64

Outliers in highwayA08:
35       21.0
53       17.0
81       15.0
109      22.0
110      21.0
         ... 
425

In [8]:
# Buscamos duplicados en todo el DataFrame
duplicates = df.duplicated()

# Contamos el número de duplicados
number_of_duplicates = duplicates.sum()

# Muestra el número de duplicados
print(f'Número de filas duplicadas: {number_of_duplicates}')

# SVemos las filas duplicadas:
duplicated_rows = df[duplicates]
print(duplicated_rows)

Número de filas duplicadas: 1100
       Year   Manufacturer              Model                       VClass  \
2269   2010  Mercedes-Benz               C300                 Compact Cars   
2345   1987      Chevrolet             Camaro              Subcompact Cars   
2387   2011            BMW  335ci Convertible              Subcompact Cars   
3014   1987        Peugeot          505 Sedan                 Compact Cars   
4060   2003         Suzuki  Vitara 4 Door 4WD  Sport Utility Vehicle - 4WD   
...     ...            ...                ...                          ...   
46145  1993          Honda            Prelude              Subcompact Cars   
46157  1993        Pontiac   Firebird/Formula              Subcompact Cars   
46165  1993          Acura              Vigor                 Compact Cars   
46181  1993       Plymouth    Sundance/Duster                 Compact Cars   
46185  1993         Subaru         Legacy AWD                 Compact Cars   

      fuelType         fuelTyp

In [9]:
# Eliminamos duplicados estrictos (donde todas las columnas relevantes son idénticas)
df = df.drop_duplicates()

# Comparamos el número de filas antes y después de eliminar duplicados para ver cuántos se han eliminado
num_rows_before = df.shape[0]
num_rows_after = df.shape[0]
num_duplicates_removed = num_rows_before - num_rows_after

num_rows_before, num_rows_after, num_duplicates_removed


(25943, 25943, 0)

## IV Estadísticas descriptivas

- Año (Year): El rango de años de modelo va de 1984 a 2024, indicando una amplia gama de vehículos desde modelos más antiguos hasta autos de este año. La media de 2007 sugiere que la mayoría de los vehículos en el dataset son relativamente recientes, con una tendencia hacia modelos más nuevos en el cuartil superior (75% de los vehículos son del año 2017 o más recientes).

- Economía de Combustible (city08, highway08, comb08): Las medias de economía de combustible en ciudad (20.86 mpg), carretera (27.50 mpg), y combinada (23.28 mpg) destacan una eficiencia moderada, con vehículos más eficientes en carretera que en ciudad.

- Economía de Combustible Alternativo (cityA08, highwayA08, combA08): Los valores promedio cercanos a 1 para estas métricas sugieren que pocos vehículos en el dataset utilizan combustibles alternativos. Los valores máximos altos reflejan la presencia de algunos vehículos altamente eficientes que utilizan combustibles alternativos.

- Emisiones de CO2 (co2, co2TailpipeGpm): La media de CO2 (-1) sugiere valores atípicos o errores en los datos, ya que el CO2 no puede ser negativo. La media del CO2 por milla de gasolina (418.84 g/mi) y su rango indican una variabilidad considerable en las emisiones entre vehículos, desde modelos muy limpios hasta otros altamente contaminantes.

La columna co2 parece estar utilizando el valor -1 para representar una falta de datos o situaciones no aplicables (posiblemente vehículos eléctricos que no emiten CO2). Por ello, es aconsejable para el análisis utilizar la columna co2TailpipeGpm que proporciona una medida más directa y, aparentemente, completa de las emisiones de CO2, con 0 representando vehículos sin emisiones directas (como vehículos eléctricos) y valores más altos reflejando mayores emisiones. 

- Rango (range): La media de rango es casi 4 millas, pero esto está sesgado por la mayoría de vehículos que no son eléctricos (indicado por la mediana de 0). Los valores máximos indican la presencia de EVs con rangos significativos, hasta 520 millas.

- Costo de Combustible (fuelCost08, fuelCostA08): Los costos anuales de combustible convencional y alternativo varían ampliamente, reflejando diferencias en la eficiencia de combustible, los tipos de combustible y los precios del combustible.


In [10]:
df.describe()

,Year,city08,highway08,comb08,cityA08,highwayA08,combA08,co2,co2TailpipeGpm,range,fuelCost08,fuelCostA08
count,25943.000000,25943.000000,25943.000000,25943.000000,25943.000000,25943.000000,25943.000000,25943.000000,25943.000000,25943.000000,25943.000000,25943.000000
mean,2007.173303,20.856455,27.502987,23.280924,1.130478,1.292564,1.187141,152.135952,418.840091,3.796824,2747.816367,100.580118
std,11.442122,12.004977,10.502401,11.269418,8.017903,8.025481,7.933359,200.484426,117.671650,31.975864,829.181855,551.623041
min,1984.000000,7.000000,9.000000,8.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,500.000000,0.000000
25%,1999.000000,16.000000,23.000000,18.000000,0.000000,0.000000,0.000000,-1.000000,355.480000,0.000000,2200.000000,0.000000
50%,2009.000000,19.000000,26.000000,21.000000,0.000000,0.000000,0.000000,-1.000000,421.000000,0.000000,2650.000000,0.000000
75%,2017.000000,22.000000,30.000000,25.000000,0.000000,0.000000,0.000000,352.000000,483.000000,0.000000,3250.000000,0.000000
max,2024.000000,153.000000,140.000000,142.000000,145.000000,121.000000,133.000000,829.000000,1110.875000,520.000000,6650.000000,4600.000000


## V Guardar DataSet

In [11]:
# Restablecer el índice del DataFrame
df.reset_index(drop=True, inplace=True)

In [12]:
# Guardar el DataFrame en un archivo CSV
df.to_csv('Clean_Vehicle_Fuel.csv', index=False)